# Import required dependencies

In [ ]:
import numpy
import cv2
import glob
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [30, 15]

# Q1.1

Discuss effects of the following parameters:
- $\sigma_{x}$
    - higher values lead to smoother images/less entropy at edges
- $\sigma_{\sigma^{'}}$
    - Lower values focus on smaller area
    - Higher values allow more effect from neighboring pixels
- $k$
    - Higher k leads to smaller gradient value
        - R is positive with high magnitude for a corner
        - R is negaitve with high magnitude for an edge
        - R is near 0 in areas with low change

In [ ]:
def generate_gradient_matrix(src, blockSize, ksize, k):

    src_size_y, src_size_x = src.shape
    scored_image_gradient = numpy.zeros(src.shape)
    gradient_x = cv2.Sobel(src, cv2.CV_32F, 1, 0, ksize=ksize)
    gradient_y = cv2.Sobel(src, cv2.CV_32F, 0, 1, ksize=ksize)
    gradient_xx = numpy.square(gradient_x)
    gradient_yy = numpy.square(gradient_y)
    gradient_xy = numpy.multiply(gradient_x, gradient_y)

    for y in range(src_size_y):
        for x in range(src_size_x):
            M = numpy.zeros((2, 2))

            ymin = int(max(0, y - blockSize / 2))
            ymax = int(min(src_size_y, y + blockSize / 2))
            xmin = int(max(0, x - blockSize / 2))
            xmax = int(min(src_size_x, x + blockSize / 2))

            for v in range(ymin, ymax):
                for u in range(xmin, xmax):
                    M[0, 0] += gradient_xx[v, u]
                    M[0, 1] += gradient_xy[v, u]
                    M[1, 1] += gradient_yy[v, u]
                    M[1, 0] = M[0, 1]

            scored_image_gradient[y, x] = numpy.linalg.det(M) - k * (numpy.trace(M) ** 2)

    return scored_image_gradient



def harris(src, blockSize, ksize, k):
    return generate_gradient_matrix(src, blockSize, ksize, k)


filepath = './homework2/objets/objet1.jpg'
bgr_image = cv2.imread(filepath)
rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
gray_blur = cv2.GaussianBlur(gray, (5, 5), 1)
gray_normalized = numpy.float32(gray_blur)
gray_normalized /= 255
features = harris(gray_normalized, 2, 3, 0.04)
cv2.imwrite('./images/q1_2_heatmap.jpg', features)

In [ ]:
rgb_image[features > 0.4 * features.max()] = [255, 0, 0]
cv2.imwrite('./images/q1_2_corners.jpg', cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR))

In [ ]:
plt.imshow(features, cmap='gray')

In [ ]:
plt.imshow(rgb_image)

In [ ]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()
template_paths = glob.glob('./homework2/objets/*.jpg')
scene_paths = glob.glob('./homework2/images_test/*.jpg')

for scene_path in scene_paths:
    
    scene_bgr_image = cv2.imread(scene_path)
    scene_rgb_image = cv2.cvtColor(scene_bgr_image, cv2.COLOR_BGR2RGB)
    scene_gray_image = cv2.cvtColor(scene_bgr_image, cv2.COLOR_BGR2GRAY)
    scene_gray_blur_image = cv2.GaussianBlur(scene_gray_image, (5, 5), 1)
    scene_keypoints, scene_descriptors = sift.detectAndCompute(scene_gray_blur_image, None)

    
    for template_path in template_paths:
        
        template_bgr_image = cv2.imread(template_path)
        template_rgb_image = cv2.cvtColor(template_bgr_image, cv2.COLOR_BGR2RGB)
        template_gray_image = cv2.cvtColor(template_bgr_image, cv2.COLOR_BGR2GRAY)
        template_gray_blur_image = cv2.GaussianBlur(template_gray_image, (5, 5), 1)
        template_keypoints, template_descriptors = sift.detectAndCompute(template_gray_image, None)
        
        knn_matches = bf.knnMatch(template_descriptors, scene_descriptors, k = 2)
        good = []
        for m, n in knn_matches:
            if m.distance < 0.8 * n.distance:
                good.append([m])


        img3 = cv2.drawMatchesKnn(template_rgb_image, template_keypoints, scene_rgb_image, scene_keypoints, good, None, flags = cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        plt.imshow(img3)
        plt.show()

In [ ]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()


template_paths = []
scene_paths = []
template_paths += glob.glob('./homework2/objets/*.jpg')
template_paths += glob.glob('./homework2/extra_objets/*.jpg')
scene_paths += glob.glob('./homework2/images_test/*.jpg')
scene_paths += glob.glob('./homework2/extra_test/*.jpg')


for scene_path in scene_paths:
    
    scene_bgr_image = cv2.imread(scene_path)
    scene_rgb_image = cv2.cvtColor(scene_bgr_image, cv2.COLOR_BGR2RGB)
    scene_gray_image = cv2.cvtColor(scene_bgr_image, cv2.COLOR_BGR2GRAY)
    scene_keypoints, scene_descriptors = sift.detectAndCompute(scene_gray_image, None)


    for template_path in template_paths:
        
        template_bgr_image = cv2.imread(template_path)
        template_rgb_image = cv2.cvtColor(template_bgr_image, cv2.COLOR_BGR2RGB)
        template_gray_image = cv2.cvtColor(template_bgr_image, cv2.COLOR_BGR2GRAY)
        template_keypoints, template_descriptors = sift.detectAndCompute(template_gray_image, None)
        
        knn_matches = bf.knnMatch(template_descriptors, scene_descriptors, k = 2)
        good = []
        for m, n in knn_matches:
            if m.distance < 0.8 * n.distance:
                good.append([m])


        img3 = cv2.drawMatchesKnn(template_rgb_image, template_keypoints, scene_rgb_image, scene_keypoints, good, None)
        plt.imshow(img3)
        plt.show()
        
        
        output_path = './q3_2_match' + str(scene_paths.index(scene_path)) + '.jpg'
        cv2.imwrite(output_path, img3)